# Performance of the euro model RSF_PR_LR 

**(Rank and SoFifa features - Prediction Results - Logistic Regression)**

In [0]:
import os

import pandas as pd
import numpy as np
from dotenv import load_dotenv

from psycopg2 import sql, connect
from sqlalchemy import create_engine, text

import datetime


In [0]:
load_dotenv()
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_HOST = os.getenv('DB_HOST')
DB_PORT = os.getenv('DB_PORT')
DB_NAME = os.getenv('DB_NAME')

DB_TN_FBREF_RESULTS = os.getenv('DB_TN_FBREF_RESULTS')
DB_TN_SOFIFA_TEAMS_STATS = os.getenv('DB_TN_SOFIFA_TEAMS_STATS')
DB_TN_MODELS_RESULTS = os.getenv('DB_TN_MODELS_RESULTS')

In [0]:
connection_url = f'postgresql+psycopg2://{(DB_USER)}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}'
engine = create_engine(connection_url)

with engine.connect() as connection:
    query_pr_results_df = text(f"SELECT * FROM {DB_TN_MODELS_RESULTS} WHERE model = 'RSF_PR_LR'")
    models_pr_results_df = pd.read_sql(query_pr_results_df, connection)
    qery_ps_results_df = text(f"SELECT * FROM {DB_TN_MODELS_RESULTS} WHERE model = 'RSF_PS_LR'")
    models_ps_results_df = pd.read_sql(qery_ps_results_df, connection)
    query_euro_matchs = text(f"SELECT * FROM {DB_TN_FBREF_RESULTS} WHERE league = 'INT-European Championships' AND season = '2425' ORDER BY date ASC")
    schedule_euro_df = pd.read_sql(query_euro_matchs, connection)


In [0]:
models_ps_results_df


In [0]:
models_pr_results_df

In [0]:
schedule_euro_df

In [0]:
# remove if game is dupplicated keep last date_inference
models_pr_results_df__keep_last_infered = models_pr_results_df.drop_duplicates(subset=['game'], keep='last')
models_ps_results_df__keep_last_infered = models_ps_results_df.drop_duplicates(subset=['game'], keep='last')

# merge on game
schedule_euro_df__merge__models_pr_results_df__keep_last_infered = pd.merge(schedule_euro_df, models_pr_results_df__keep_last_infered, on='game', how='left')

schedule_euro_df__merge__models_pr_results_df__keep_last_infered

In [0]:
schedule_euro_df__merge__models_pr_results_df__keep_last_infered.columns

In [0]:
schedule_euro_df__merge__models_pr_results_df__keep_last_infered.loc[:, 'winner'] = schedule_euro_df__merge__models_pr_results_df__keep_last_infered.apply(lambda x: 1 if x['home_g'] > x['away_g'] else -1 if x['home_g'] < x['away_g'] else 0 if x['home_g'] == x['away_g'] else None, axis=1)
schedule_euro_df__merge__models_pr_results_df__keep_last_infered.loc[:, 'winner_pred'] = schedule_euro_df__merge__models_pr_results_df__keep_last_infered.apply(lambda x: 1 if x['prob_home_win'] > x['prob_away_win'] and x['prob_home_win'] > x['prob_draw'] else -1 if x['prob_away_win'] > x['prob_home_win'] and x['prob_away_win'] > x['prob_draw'] else 0 if x['prob_draw'] > x['prob_home_win'] and x['prob_draw'] > x['prob_away_win'] else None, axis=1)


In [0]:
schedule_euro_df__merge__models_pr_results_df__keep_last_infered[['date', 'home_team_x', 'away_team_x',
 'home_g', 'away_g', 'prob_home_win', 'prob_draw', 'prob_away_win', 'winner', 'winner_pred']].tail(15)

In [0]:
accuracy_winner = schedule_euro_df__merge__models_pr_results_df__keep_last_infered[
    (schedule_euro_df__merge__models_pr_results_df__keep_last_infered['winner'] == schedule_euro_df__merge__models_pr_results_df__keep_last_infered['winner_pred']) &
    (schedule_euro_df__merge__models_pr_results_df__keep_last_infered['winner'].isnull() == False)
    ].shape[0] / schedule_euro_df__merge__models_pr_results_df__keep_last_infered[
        (schedule_euro_df__merge__models_pr_results_df__keep_last_infered['winner'].isnull() == False)].shape[0]
print(f"accuracy_winner = {accuracy_winner}")

In [0]:
schedule_euro_df__merge__models_pr_results_df__keep_last_infered[
    (schedule_euro_df__merge__models_pr_results_df__keep_last_infered['winner'] == schedule_euro_df__merge__models_pr_results_df__keep_last_infered['winner_pred']) &
    (schedule_euro_df__merge__models_pr_results_df__keep_last_infered['winner'].isnull() == False)
    ].shape[0]

In [0]:
schedule_euro_df__merge__models_pr_results_df__keep_last_infered[
        (schedule_euro_df__merge__models_pr_results_df__keep_last_infered['winner'].isnull() == False)].shape[0]

In [0]:
schedule_euro_df__merge__models_pr_results_df__keep_last_infered

In [0]:
models_ps_results_df__keep_last_infered__col_drop = models_ps_results_df__keep_last_infered[['game', 'prob_home_team_score', 'prob_away_team_score']]
schedule_euro_df__merge__models_pr_results_df__keep_last_infered__merge__models_ps_results_df__keep_last_infered__col_drop = pd.merge(schedule_euro_df__merge__models_pr_results_df__keep_last_infered, models_ps_results_df__keep_last_infered__col_drop, on='game', how='left')



In [0]:
schedule_euro_df__merge__models_pr_results_df__keep_last_infered__merge__models_ps_results_df__keep_last_infered__col_drop.columns

In [0]:
schedule_euro_df__merge__models_pr_results_df__keep_last_infered__merge__models_ps_results_df__keep_last_infered__col_drop[['date', 'home_team_x', 'away_team_x',
 'home_g', 'away_g', 'prob_home_team_score_y', 'prob_away_team_score_y', 'prob_home_win', 'prob_draw', 'prob_away_win', 'winner', 'winner_pred']].tail(15)